# Resources

1. https://bionerdnotes.wordpress.com/2020/03/25/how-to-use-modeller-for-homology-based-modelling/
2. https://bionerdnotes.wordpress.com/2020/04/01/multi-chain-modelling-in-modeller/
3. https://bionerdnotes.wordpress.com/2019/05/25/secondary-structure-prediction/

In [2]:
from modeller import *
from pathlib import Path
import os

# log.verbose()
env = Environ()

data = Path("./")/'..'/'data'/'structure_search'
data.mkdir(exist_ok=True)


# Read in the sequence database

Download from https://salilab.org/modeller/supplemental.html

In [ ]:
# !wget https://salilab.org/modeller/downloads/pdb_95.pir.gz -o pdb_95.pir.gz
# !gunzip pdb_95.pir.gz
# !mv pdb_95.pir data/structure_search/

In [5]:
sdb = SequenceDB(env)

if not os.path.exists(data/'pdb_95.bin'):
    #-- Read in the sequence database    
    sdb.read(seq_database_file=str(data/'pdb_95.pir'), seq_database_format='PIR', chains_list='ALL', minmax_db_seq_len=(30, 4000), clean_sequences=True)

    #-- Write the sequence database in binary form
    sdb.write(seq_database_file=str(data/'pdb_95.bin'), seq_database_format='BINARY',
            chains_list='ALL')
else:
    #-- Now, read in the binary database
    sdb.read(seq_database_file=str(data/'pdb_95.bin'), seq_database_format='BINARY',
            chains_list='ALL')



SEQ_DATABASE_FILE         : ../data/structure_search/pdb_95.bin
SEQ_DATABASE_FORMAT       : BINARY
CHAINS_LIST               : ALL
CLEAN_SEQUENCES           :            T
MINMAX_DB_SEQ_LEN         :            0      999999
Number of sequences       :        79268
Number of residues        :     20470869
Length of longest sequence:         3725



## Alignment Object:

- Represents an explicit sequence alignment between two or more sequences or structures.
- Contains all aligned sequences with gaps and their positional relationships.
- Used to construct models (directly guides model building in comparative modeling).


## Profile Object:

- A more abstract representation derived from an alignment or a single sequence.
- Stores residue probabilities or frequencies for each position in a sequence.
- More compact and faster for database searches or iterations like PSI-BLAST.
- Used in prf.build to search databases and identify potential homologs.

In [6]:
#-- Read in the target sequence/alignment
# https://salilab.org/modeller/manual/node296.html

aln = Alignment(env)
aln.append(file=str(data/'Assign_3.ali'), alignment_format='pir', align_codes='all')

#-- Convert the input sequence/alignment into profile format
prf = aln.to_profile()


`sdb`: Input database object (sequence_db) containing sequences for searching.

`matrix_offset=-450`: Adjusts scoring offsets for the similarity matrix, influencing match/mismatch scores.

`rr_file='${LIB}/blosum62.sim.mat'`: Specifies the similarity matrix file used (BLOSUM62 here). `${LIB}` is a placeholder for the MODELLER library path, which is typically set up during installation.

`gap_penalties_1d=(-500, -50)`: Penalties for opening and extending gaps in the alignment.

`n_prof_iterations=1`: Specifies a single iteration of profile generation (no PSI-BLAST-like expansion).

`check_profile=False`: Disables internal checks for convergence or errors in profile iterations.

`max_aln_evalue=0.01`: Only retains alignments with an E-value ≤ 0.01, filtering weaker matches.

The `Profile.build()` command has many options. In this example **rr_file** is set to use the _BLOSUM62 similarity matrix_ (file `"blosum62.sim.mat"` provided in the MODELLER distribution). Accordingly, the parameters **matrix_offset** and **gap_penalties_1d** are set to the appropriate values for the BLOSUM62 matrix.

For this example, we will run only one search iteration by setting the parameter **n_prof_iterations** equal to _1_. Thus, there is no need for checking the profile for deviation (**check_profile** set to _False_).

Finally, the parameter **max_aln_evalue** is set to _0.01_, indicating that only sequences with e-values smaller than or equal to 0.01 will be included in the final profile.

### 1. What Happens if the Search Iteration is More than 1?
If **`n_prof_iterations > 1`**, the process becomes iterative, similar to PSI-BLAST. Here's how it works:
- After the first iteration, a **profile** is created based on the aligned sequences from the database.
- In subsequent iterations, the profile is used to search the sequence database again, potentially identifying more distantly related homologs.
  
This iterative approach can improve results by:
- **Enhancing sensitivity**: The profile captures conserved patterns across related sequences, allowing it to detect homologs that might be missed in a single iteration.
- **Refining alignments**: Subsequent iterations adjust the profile to better represent the sequence's evolutionary context.

However, more iterations can also:
- Increase runtime significantly.
- Risk adding false positives if unrelated sequences are weakly similar to parts of the profile.

---

### 2. What Does it Mean to "Check for Profile Deviation"?
- **Checking for profile deviation** means evaluating how much the profile changes between iterations. It ensures that:
  - The profile is converging to a stable representation.
  - The alignment and detected sequences are meaningful and not diverging or becoming less specific.

**How to Check for Deviation:**
- Compare the profile content after each iteration. If the new profile matches well with the previous one (e.g., similarity scores stabilize), the profile has converged.
- MODELLER can automatically perform this check if `check_profile=True`, but it is often unnecessary for a single iteration (`n_prof_iterations=1`).

---

### 3. What Does "E-value" Mean and Why ≤ 0.01?
The **E-value** (expected value) is a statistical measure used in sequence alignment to estimate the number of matches expected to occur by chance in a database search. It is calculated based on:
- The length of the query and database sequences.
- The alignment score.

**Key Points:**
- **Lower E-value = Higher confidence**: Smaller E-values indicate the alignment is less likely to have occurred by random chance.
- **E-value of 0.01**: Indicates that only 1 match out of 100 would be expected to occur by chance. This threshold ensures you retain high-confidence homologs.

**Why Use E-values ≤ 0.01?**
- For comparative modeling, homologs with very low E-values are likely to have meaningful structural similarities.
- Filtering out weaker matches reduces noise and avoids aligning irrelevant sequences that may introduce errors in the model.

In [7]:
#-- Scan sequence database to pick up homologous sequences
prf.build(sdb, matrix_offset=-450, rr_file='${LIB}/blosum62.sim.mat',
          gap_penalties_1d=(-500, -50), n_prof_iterations=1,
          check_profile=False, max_aln_evalue=0.01)


profile_iteration_> processing sequence:       1    759      1     0.0400000     0.0400000     0.0400000     1
profile_iteration_> processing sequence:    7928    759      1    10.5000000     0.0013244     0.0000051   755
profile_iteration_> processing sequence:   15855    759      1    20.6399994     0.0013018     0.0000050   768
profile_iteration_> processing sequence:   23782    759      1    30.9099998     0.0012997     0.0000051   769
profile_iteration_> processing sequence:   31709    759      1    41.0200005     0.0012936     0.0000050   773
profile_iteration_> processing sequence:   39636    759      1    51.2999992     0.0012943     0.0000050   773
profile_iteration_> processing sequence:   47563    759      1    61.4799995     0.0012926     0.0000050   774
profile_iteration_> processing sequence:   55490    759      1    71.7799988     0.0012936     0.0000050   773
profile_iteration_> processing sequence:   63417    759      1    82.1399994     0.0012952     0.0000050   772
p

In [8]:
#-- Write out the profile in text format
prf.write(file=str(data/'build_profile.prf'), profile_format='TEXT')

#-- Convert the profile back to alignment format
aln = prf.to_alignment()

#-- Write out the alignment file
aln.write(file=str(data/'build_profile.ali'), alignment_format='PIR')